In [1]:
# py imports
import os, sys
import glob

# third party
import tensorflow as tf
import scipy.io as sio
import numpy as np
from keras.backend.tensorflow_backend import set_session
from scipy.interpolate import interpn

Using TensorFlow backend.


# Setup Project

In [2]:
sys.path.append('../src') # add source
sys.path.append('../ext/medipy-lib')
import medipy
import networks
from medipy.metrics import dice
import datagenerators

In [10]:
vol_size=(160,192,224)
nf_enc=[16,32,32,32]
nf_dec=[32,32,32,32,32,16,16,3]
model_name = 'demo'
iter_num = 0

In [4]:
# Anatomical labels we want to evaluate
labels = sio.loadmat('../data/labels.mat')['labels'][0]

In [5]:
atlas = np.load('../data/atlas_norm.npz')
atlas_vol = atlas['vol']
atlas_seg = atlas['seg']
atlas_vol = np.reshape(atlas_vol, (1,)+atlas_vol.shape+(1,))

In [8]:
from itkwidgets import view
view(atlas_seg)

Viewer(image=<itkImagePython.itkImageF3; proxy of <Swig Object of type 'itkImageF3 *' at 0x7fcc1c1526f0> >)

In [12]:
net = networks.unet(vol_size, nf_enc, nf_dec)
try:
    net.load_weights('../models/' + model_name +
                     '/' + str(iter_num) + '.h5')
except Exception as e:
    print('No weights could be loaded:', e)

No weights could be loaded: Unable to open file (unable to open file: name = '../models/demo/0.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)


In [13]:
xx = np.arange(vol_size[1])
yy = np.arange(vol_size[0])
zz = np.arange(vol_size[2])
grid = np.rollaxis(np.array(np.meshgrid(xx, yy, zz)), 0, 4)

X_vol, X_seg = datagenerators.load_example_by_name('../data/test_vol.npz', '../data/test_seg.npz')


# Run the Prediction

In [ ]:
%%time
pred = net.predict([X_vol, atlas_vol])

In [1]:
# Warp segments with flow
flow = pred[1][0, :, :, :, :]
sample = flow+grid
sample = np.stack((sample[:, :, :, 1], sample[:, :, :, 0], sample[:, :, :, 2]), 3)
warp_seg = interpn((yy, xx, zz), X_seg[0, :, :, :, 0], sample, method='nearest', bounds_error=False, fill_value=0)

vals, _ = dice(warp_seg, atlas_seg, labels=labels, nargout=2)
print(np.mean(vals), np.std(vals))

NameError: name 'pred' is not defined